In [61]:
import os 
import pandas as pd 
import json 
import yaml 
import shutil

This is the way to prepare separate jsons for every data type (impression, findings, target).

In another ipynb, we have v2: where each json contains the impression, findings, and target, and the Dataset code selects which one to keep. 

In [ ]:
# these are the files in raw_data 
# mimic_test_reports_new.csv  
# mimic_train_reports_new.csv  
# mimic_train_subset_reports_new.csv 
# mimic_val_subset_reports_new.csv

In [86]:
# this is the path to the codebase 
lavis_root = '/n/data1/hms/dbmi/rajpurkar/lab/home/kt220/SPR23/LAVIS-kttian-VIT'

# this is an example dataset yaml config relative path (relative to lavis_root)
# dataset_config = 'lavis/configs/datasets/mimic_cxr/sepna_wval_findings_full_224.yaml'

# this is the path to where all the dataset jsons live 
cache_root = '/n/data1/hms/dbmi/rajpurkar/lab/home/kt220/SPR23/export/home/.cache/lavis'

# this is an example dataset json relative math (relative to cache_root)
# mimic_cxr_sepna/annotations/mimic_test_findings_full_wval.json

# this is the local folder where the dataset jsons are saved 
out_root = 'json_data'


In [87]:
# we plan to copy 'json_data' to FINAL_JSON_DIR
subfolder = 'mimic_cxr_0529'
FINAL_JSON_DIR = os.path.join(cache_root, subfolder, 'annotations')
# we plan to copy 'dataset_configs' to FINAL_YAML_DIR
FINAL_YAML_DIR = os.path.join(lavis_root, 'lavis/configs/datasets', subfolder)
if not os.path.exists(FINAL_JSON_DIR):
    os.makedirs(FINAL_JSON_DIR)
if not os.path.exists(FINAL_YAML_DIR):
    os.makedirs(FINAL_YAML_DIR)
print(FINAL_JSON_DIR)
print(FINAL_YAML_DIR)

/n/data1/hms/dbmi/rajpurkar/lab/home/kt220/SPR23/export/home/.cache/lavis/mimic_cxr_0529/annotations
/n/data1/hms/dbmi/rajpurkar/lab/home/kt220/SPR23/LAVIS-kttian-VIT/lavis/configs/datasets/mimic_cxr_0529


Create the data jsons 

In [88]:
def get_report(row, report_mode):
    if report_mode == 'findings':
        return row[report_mode]
    elif report_mode == 'impression':
        return row[report_mode]
    else: # both
        # TODO: what is the proper way to generate both?
        return row['target']
        # return f"FINDINGS: {row['findings']} \n IMPRESSION: {row['impression']}"

def process_data(data_df, report_mode):
    if report_mode == 'findings' or report_mode == 'impression':
        return data_df.dropna(subset=[report_mode])
    else:
        return data_df.dropna()

In [89]:
def create_json(report_mode='impression', handle_val='wval', size_mode='subset', 
                img_ext='png', save=False, ):
    subset_sizes = {'train': 128, 'val': 32, 'test': 32}

    def convert_to_json_item(row, report_mode=report_mode):
        ''' 
        assumes input is a row of the dataframe
        ''' 
        dicom_id = row['dicom_id']
        subject_id = f"p{str(int(row['subject_id']))}"
        study_id = f"s{str(int(row['study_id']))}"
        img_file = f"{subject_id[:3]}/{subject_id}/{study_id}/{dicom_id}." + img_ext 
        report = row[report_mode]
        image_id = "".join([str(int(x, 16)) for x in dicom_id.split('-')])
        return {'image': img_file, 'caption': report, 'dicom_id': dicom_id, 
                'subject_id': subject_id, 'study_id': study_id, 'image_id': image_id}
    # load data
    raw_filename = {}
    raw_filename['test'] = os.path.join('raw_data', 'mimic_test_reports_new.csv')
    if handle_val == 'wval':
        raw_filename['train'] = os.path.join('raw_data', 'mimic_train_subset_reports_new.csv')
        raw_filename['val'] = os.path.join('raw_data', 'mimic_val_subset_reports_new.csv')
    elif handle_val == 'noval':
        raw_filename['train'] = os.path.join('raw_data', 'mimic_train_reports_new.csv')
        raw_filename['val'] = os.path.join('raw_data', 'mimic_val_subset_reports_new.csv')

    
   
    # create annotations 
    for split in ['train', 'val', 'test']:
        print('-'*40)
        print(split)
        size = subset_sizes[split]
        data_df = pd.read_csv(raw_filename[split])
        
        # create file path of where to save annotations json 
        outfile = os.path.join(out_root, f'mimic_cxr_instruct_{report_mode}_{handle_val}_{size_mode}_{split}.json')
        
        # prepare data df
        # TODO: change data processing / dropna for full data later
        data_split = process_data(data_df, report_mode)
        if size_mode == 'subset':
            data_split = data_split[:size]

        # loop through df and create json data 
        apply_output = data_split.apply(convert_to_json_item, axis=1)
        json_data = list(apply_output)

        if split == 'val' or split == 'test':
            for item in json_data:
                item['caption'] = [item['caption']]
        
        if save:
            with open(outfile, "w") as f:
                json.dump(json_data, f)
            print(f"saved to {outfile}, len {len(json_data)}")
        else:
            print(f"len {len(json_data)}")
            print("potential outfile:", outfile)
            print(json_data[0])

In [90]:
for img_size in [224, 364]:
    for report_mode in ['findings', 'impression', 'target']:
        for handle_val in ['wval', 'noval']:
            for size_mode in ['subset', 'full']:
                create_json(report_mode=report_mode, handle_val=handle_val, size_mode=size_mode, 
                            img_ext='png', save=True)

----------------------------------------
train
saved to json_data/mimic_cxr_instruct_findings_wval_subset_train.json, len 128
----------------------------------------
val
saved to json_data/mimic_cxr_instruct_findings_wval_subset_val.json, len 32
----------------------------------------
test
saved to json_data/mimic_cxr_instruct_findings_wval_subset_test.json, len 32
----------------------------------------
train
saved to json_data/mimic_cxr_instruct_findings_wval_full_train.json, len 260159
----------------------------------------
val
saved to json_data/mimic_cxr_instruct_findings_wval_full_val.json, len 2026
----------------------------------------
test
saved to json_data/mimic_cxr_instruct_findings_wval_full_test.json, len 3518
----------------------------------------
train
saved to json_data/mimic_cxr_instruct_findings_noval_subset_train.json, len 128
----------------------------------------
val
saved to json_data/mimic_cxr_instruct_findings_noval_subset_val.json, len 32
----------

In [4]:
# for img_size in [224, 364]:
#     for report_mode in ['findings', 'impression']:
#         for handle_val in ['wval', 'noval']:
#             for size_mode in ['subset', 'full']:
#                 create_json(report_mode=report_mode, handle_val=handle_val, size_mode=size_mode, 
#                             img_ext='png', save=True)

Create the data config yamls & print out the builder code

In [94]:
# img_size = 224
# report_mode = 'findings'
# handle_val = 'wval'
# size_mode = 'subset'

def create_yaml(report_mode='impression', handle_val='wval', size_mode='subset', img_size=224):
    registry_name = f'mimic_cxr_instruct_{report_mode}_{handle_val}_{size_mode}_{img_size}'
    yaml_obj = {} 
    yaml_obj['datasets'] = {registry_name: {}}
    yaml_obj['datasets'][registry_name]['data_type'] = 'images'
    yaml_obj['datasets'][registry_name]['build_info'] = {}

    ann_dict = {}
    for split in ['train', 'val', 'test']:
        # toy url: f'https://storage.googleapis.com/sfr-vision-language-research/datasets/coco_karpathy_{split}.json'
        ann_dict[split] = {'url': '', 
                        'storage': f'{subfolder}/annotations/mimic_cxr_instruct_{report_mode}_{handle_val}_{size_mode}_{split}.json'}
    yaml_obj['datasets'][registry_name]['build_info']['annotations'] = ann_dict 
    if img_size == 224:
        yaml_obj['datasets'][registry_name]['build_info']['images'] = {'storage': 'mimic_cxr/images_224'}
    else:
        yaml_obj['datasets'][registry_name]['build_info']['images'] = {'storage': 'mimic_cxr/images'}
    
    with open(os.path.join('dataset_configs', f'{registry_name}.yaml'), 'w') as file:
        documents = yaml.dump(yaml_obj, file)
    
    class_name = f'MimicCxrInstruct{report_mode.capitalize()}{handle_val.capitalize()}{size_mode.capitalize()}{img_size}Builder'
    code = f'@registry.register_builder("{registry_name}")\n'
    code += f'class {class_name}(BaseDatasetBuilder):\n'
    code += f'    train_dataset_cls = ReportGenInstructDataset\n'
    code += f'    eval_dataset_cls = ReportGenInstructEvalDataset\n'
    code += f'    \n'
    code += f'    DATASET_CONFIG_DICT = {{\n'
    code += f'        "default": "configs/datasets/{subfolder}/{registry_name}.yaml",\n'
    code += f'    }}\n'
    print(code)

In [95]:
for img_size in [224, 364]:
    for report_mode in ['target', 'findings', 'impression']:
        for handle_val in ['wval', 'noval']:
            for size_mode in ['subset', 'full']:
                create_yaml(report_mode=report_mode, handle_val=handle_val, size_mode=size_mode, img_size=img_size)

@registry.register_builder("mimic_cxr_instruct_target_wval_subset_224")
class MimicCxrInstructTargetWvalSubset224Builder(BaseDatasetBuilder):
    train_dataset_cls = ReportGenInstructDataset
    eval_dataset_cls = ReportGenInstructEvalDataset
    
    DATASET_CONFIG_DICT = {
        "default": "configs/datasets/mimic_cxr_0529/mimic_cxr_instruct_target_wval_subset_224.yaml",
    }

@registry.register_builder("mimic_cxr_instruct_target_wval_full_224")
class MimicCxrInstructTargetWvalFull224Builder(BaseDatasetBuilder):
    train_dataset_cls = ReportGenInstructDataset
    eval_dataset_cls = ReportGenInstructEvalDataset
    
    DATASET_CONFIG_DICT = {
        "default": "configs/datasets/mimic_cxr_0529/mimic_cxr_instruct_target_wval_full_224.yaml",
    }

@registry.register_builder("mimic_cxr_instruct_target_noval_subset_224")
class MimicCxrInstructTargetNovalSubset224Builder(BaseDatasetBuilder):
    train_dataset_cls = ReportGenInstructDataset
    eval_dataset_cls = ReportGenInstructEvalD

Move the local folders to final folders

In [96]:
shutil.copytree('json_data', FINAL_JSON_DIR, dirs_exist_ok=True)
shutil.copytree('dataset_configs', FINAL_YAML_DIR, dirs_exist_ok=True)

'/n/data1/hms/dbmi/rajpurkar/lab/home/kt220/SPR23/LAVIS-kttian-VIT/lavis/configs/datasets/mimic_cxr_0529'